In [7]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import cohere
import pinecone
from pinecone import Pinecone
from pinecone import ServerlessSpec

In [8]:
#setting up my cohere API using API key
co = cohere.Client("kiPt2Uk4mEr2UxymdrofzBYblAGXZY5LF2u0FoQB")

In [9]:
#loading my amazon books datasets
book_data = pd.read_json('amazon_books_data.json')

In [10]:
#info about my datasets
book_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159 entries, 0 to 2158
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  2159 non-null   int64  
 1   name                2159 non-null   object 
 2   summary             1919 non-null   object 
 3   category            2159 non-null   object 
 4   book_type           2159 non-null   object 
 5   production_date     2159 non-null   object 
 6   authors             2159 non-null   object 
 7   hard_cover_price    2159 non-null   object 
 8   paper_back_price    1883 non-null   object 
 9   book_rating         2159 non-null   float64
 10  total_user_ratings  2159 non-null   object 
 11  number_of_pages     2159 non-null   int64  
 12  language            2159 non-null   object 
 13  reviews             2159 non-null   object 
dtypes: float64(1), int64(2), object(11)
memory usage: 236.3+ KB


In [11]:
#removing duplicate names in my datasets
book_data = book_data.drop_duplicates('name')

In [12]:
#duplicate books in the datasets have been removed
book_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1748 entries, 0 to 2158
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1748 non-null   int64  
 1   name                1748 non-null   object 
 2   summary             1549 non-null   object 
 3   category            1748 non-null   object 
 4   book_type           1748 non-null   object 
 5   production_date     1748 non-null   object 
 6   authors             1748 non-null   object 
 7   hard_cover_price    1748 non-null   object 
 8   paper_back_price    1515 non-null   object 
 9   book_rating         1748 non-null   float64
 10  total_user_ratings  1748 non-null   object 
 11  number_of_pages     1748 non-null   int64  
 12  language            1748 non-null   object 
 13  reviews             1748 non-null   object 
dtypes: float64(1), int64(2), object(11)
memory usage: 204.8+ KB


In [13]:
#converting all my columns into strings
book_data = book_data.astype(str)

In [14]:
book_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1748 entries, 0 to 2158
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  1748 non-null   object
 1   name                1748 non-null   object
 2   summary             1748 non-null   object
 3   category            1748 non-null   object
 4   book_type           1748 non-null   object
 5   production_date     1748 non-null   object
 6   authors             1748 non-null   object
 7   hard_cover_price    1748 non-null   object
 8   paper_back_price    1748 non-null   object
 9   book_rating         1748 non-null   object
 10  total_user_ratings  1748 non-null   object
 11  number_of_pages     1748 non-null   object
 12  language            1748 non-null   object
 13  reviews             1748 non-null   object
dtypes: object(14)
memory usage: 204.8+ KB


In [15]:
book_data.head()

,id,name,summary,category,book_type,production_date,authors,hard_cover_price,paper_back_price,book_rating,total_user_ratings,number_of_pages,language,reviews
0,1,The Moth Presents: A Point of Beauty: True Sto...,NEW YORK TIMES,Literature & Fiction,Hardcover,"March 19, 2024","The Moth, Mike Birbiglia",$18.60,$13.99,4.6,95,400,English,"This book, taken from the Moth Radio program, ..."
1,2,Latitude Adjustments: Jimmy Buffett Quotes for...,Latitude Adjustments: Jimmy Buffett Quotes for...,Self-Help,Hardcover,"August 30, 2024",H. R. Gordon,$25.00,$16.00,4.0,2,10,222 pages,Calling all Parrotheads! 5 fins up for this on...
2,3,Gnomes,An updated and revised edition of the,Arts & Photography,Hardcover,"September 5, 2023","Wil Huygen, Rien Poortvliet, Brian Froud",$22.49,None,4.9,146,224,English,The book has fantastic pictures and detail. I ...
3,4,The French Ingredient: Making a Life in Paris ...,The inspiring and delicious memoir of an Ameri...,"Cookbooks, Food & Wine",Hardcover,"April 9, 2024",Jane Bertch,$19.39,$1.99,4.6,247,304,English,"Jane Berth is an engaging writer, charming sto..."
4,5,You Never Know: A Memoir,***THE INSTANT,Biographies & Memoirs,Hardcover,"May 7, 2024","Tom Selleck, Ellis Henican",$17.39,$13.99,4.6,"2,137",352,English,Let's face it. Everyone enjoys Tom Selleck's ...


In [16]:
# Function to create a descriptive sentence from the specified columns
def prepare_embedding_text(row):
    return (f"{row['name']} was written by {row['authors']} "
            f"and was produced on {row['production_date']}. "
            f"It falls under the {row['category']} category. "
            f"The hard cover price is {row['hard_cover_price']}. "
            f"It has a book rating of {row['book_rating']} based on {row['total_user_ratings']} user ratings "
            f"and contains {row['number_of_pages']} pages. "
            f"Reviews: {row['reviews']}")

# Apply the function to create a new column with the formatted text
book_data['embedding_text'] = book_data.apply(prepare_embedding_text, axis=1)

In [17]:
#checking my sentences to be embedded by cohere ai model
book_data['embedding_text'][67]

#wow, I'm impressed

"We Solve Murders: A Novel was written by Richard Osman and was produced on September 17, 2024. It falls under the Mystery, Thriller & Suspense category. The hard cover price is $26.29. It has a book rating of 4.5 based on 1,147 user ratings and contains 400 pages. Reviews: If you love The Thursday Murder Club you will love this book.  Lots of really, really great wit.  As I am not British I am sure a few quips went over my head, but there are so many, it just didn’t matter. Great murder mystery, great characters, wonderful human insight and an all around fun read.  I read a book a day - well written prose and a compelling story are hard to find.  And, it makes you laugh and keeps you on your toes.  Please write ten more quickly - as Rosie would say “chop, chop”, hmmm, that would be a great name of a book., Fans of the Thursday Murder Club will thoroughly enjoy this new series, which is only very nearly almost not quite as perfect as TMC is. Another round of Steve and Amy and Rosie mig

In [18]:
# Converting the embedding_text Series to a list
texts_to_embed = book_data['embedding_text'].tolist()

In [19]:
#Embedding my sentences
embeds = co.embed(
    texts= texts_to_embed,
    model='embed-multilingual-v3.0',
    input_type='search_document'
).embeddings


In [20]:
#shape of the newly embedded texts
shape = np.array(embeds).shape
print(shape)

(1748, 1024)


In [24]:
# initializing connection to pinecone
pc = Pinecone(api_key='e9fd2f61-5bf0-42e2-9011-9027c377d951')

index_name = 'cohere-pinecone-amazon-books'

# if the index does not exist, we create it
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=shape[1],
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# connect to index
index = pc.Index(index_name)

In [25]:
# Pinecone expects us to provide a list of tuples in the format (id, vector, metadata)
batch_size = 128

ids = [str(i) for i in range(shape[0])]
# create list of metadata dictionaries
meta = [{'text': text} for text in texts_to_embed]

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, embeds, meta))

for i in range(0, shape[0], batch_size):
    i_end = min(i+batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

# let's view the index statistics
print(index.describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1748}},
 'total_vector_count': 1748}


In [26]:
query = "Do you have book that talks about cooking"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='embed-multilingual-v3.0',
    input_type='search_query'
).embeddings

print(np.array(xq).shape)

# query, returning the top 5 most similar results
res = index.query(vector=xq, top_k=10, include_metadata=True)

(1, 1024)


In [27]:
book_knowledge = []
for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")
    book_knowledge.append(match['metadata']['text'])

0.63: Gordon Ramsay's Home Cooking: Everything You Need to Know to Make Fabulous Food was written by Gordon Ramsay and was produced on April 9, 2013. It falls under the Cookbooks, Food & Wine category. The hard cover price is $17.69. It has a book rating of 4.8 based on 5,748 user ratings and contains 320 pages. Reviews: These recipes are very simple, elegant, and delicious! I love the explanation for a good set of knives and kitchen equipment you actually need and not all the fluff they try to sell you. I look forward to making more recipes from here. Great edition to my slim cookbook collection., I didn't know until after I bought this book and read other reviews that this cookbook has essentially a companion cooking tv series that aired in the UK under the same name as the UK version of this cookbook--Ultimate Cookery Course. He really breaks everything down so easy, so understandable so that you feel like I can really take on these recipes and his food and do it justice., Having be

In [28]:
#Integrating my chatbot using already pretrained model from Cohere AI
prompt = f"""
You're a book recommendation assistant, and always open to welcome users when they really don't know what you are and help in putting them through. Your book knowledge encompasses the following books genre
'Literature & Fiction', 'Self-Help', 'Arts & Photography','Cookbooks, Food & Wine', 'Biographies & Memoirs','Humor & Entertainment', 'Reference', 'Politics & Social Sciences',
'History', "Children's Books", 'Crafts, Hobbies & Home','Computers & Technology', 'Engineering & Transportation','LGBTQ+ Books', 'Mystery, Thriller & Suspense', 'Medical Books',
'Science Fiction & Fantasy', 'Health, Fitness & Dieting','Science & Math', 'Business & Money', 'Christian Books & Bibles','Religion & Spirituality', 'Romance', 'Teen & Young Adult',
'Travel', 'Comics & Graphic Novels', 'Sports & Outdoors', 'Law','Parenting & Relationships'

## Instructions
Below is a list of book summaries and reviews related to the user's query. Each entry includes information such as book name, authors, production date, category, hard cover price, book rating, user ratings, number of pages, and a selection of user reviews.

The user's query is: "{query}"

Please provide a detailed response to the user's query based on the given information. The response should be engaging and informative, synthesizing the key points from the book summaries and reviews. Use natural language and feel free to add helpful recommendations or insights.

## Book Summaries and Reviews:
{book_knowledge}

## Approach to satisfying Users
1. Greet them nicely, and introduce yourself as 'book recommendations assistant' if the query does not address the need for a book recommendation or suggestion
2. scan through the queries thoughroughly and acknowledge the users demands
3. based on the insights from the book knowledge, try as much as possible to give the user yuour best take and address the users query
4. In a case whereby the users query is not contained in the book knowledge, or genre of books you are conversant with, you can briefly give the user a tour on what you offer and suggest something nice or close to users request
5. don't be forward to give a book reccomendation if you are not asked in the query, an introduction of what you can offer is enough please and politely ask what they will lie to get, you can briefly walk them through list of genres you cover.
6. For a query that addresses a need for a book, it's obvious such user alreadyknows what you can offer, so you can just acknowledge the users query and then skip introducing you are a book recommendation assistant

## Example Output
- High-level response: <summary based on the query>
  * <insight 1>
  * <insight 2>
  * <insight 3>
"""

response = co.chat(
  model="command-r-plus-08-2024",
  message= prompt
)

print(response.text)

- High-level response: I have several books that focus on cooking and culinary adventures, offering a delightful blend of recipes, cultural insights, and engaging narratives.

  * **Gordon Ramsay's Home Cooking: Everything You Need to Know to Make Fabulous Food:** This book is a comprehensive guide to cooking by celebrity chef Gordon Ramsay. It includes simple and elegant recipes, tips on essential kitchen equipment, and personal anecdotes. With a 4.8 rating, it's a popular choice for home cooks looking to elevate their skills.
  * **This Is a Book About Noodles:** Brendan Pang's cookbook explores the art of noodle-making and various Asian dishes. It's a great resource for noodle enthusiasts, offering a 4.7 rating and user-friendly recipes.
  * **Feeding Hannibal: A Connoisseur's Cookbook:** This unique cookbook is inspired by the TV show Hannibal, featuring exquisite recipes and culinary artistry. It includes beautiful illustrations, anecdotes, and a 4.9 rating, making it a must-have 

In [ ]:
book_data['category'].unique()